# Linear Regression (Ordinary Least Squares)

## 1. Linear Regression on Boston Housing Price Data

- 본 노트북 코드에서는 scikit-learn 오픈소스 라이브러리에서 제공하는 "보스턴 주택 가격" 데이터를 이용하여 선형 회귀 모델을 만들어보고자 합니다.
- 데이터를 불러오는데에는 sklearn.datasets 모듈, 데이터를 살펴보는 것은 pandas, 회귀(regression) 모델을 훈련하고 평가 및 예측에는 scikit-learn 패키지를 사용하겠습니다.

### (1) Prepare the dataset
- "보스턴 주택 가격" 데이터는 회귀분석 예제로 흔히 사용되는 데이터로 UCI 데이터베이스에서 제공하고 있습니다.  
- 해당 데이터는 딕셔너리(dictionary) 형태로 제공되고 있으며, pandas를 이용하여 데이터를 아래와 같이 데이터프레임(DataFrame) 형태로 저장할 수 있습니다.

In [ ]:
import pandas as pd
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data"
col_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
housing_df = pd.read_csv(url, delim_whitespace=True, names=col_names)

In [ ]:
housing_df

In [ ]:
# X와 y 나누기
X = housing_df.drop(columns=['MEDV'])  # 독립 변수들
y = housing_df['MEDV']  # 종속 변수 (목표 값)

In [ ]:
# 데이터의 크기 확인
print(X.shape, y.shape)

### (2) Split the data into training and test sets
- 선형 회귀 모델 훈련을 위해, 훈련에 사용하는 training data 와 모델 평가 및 예측에 사용하는 test data 로 나누겠습니다.
- scikit-learn 의 train_test_split 함수를 이용하여 일정한 비율로 데이터를 나눌 수 있습니다.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### (3) Define and Train a linear regression model
- 위를 통해 데이터 준비가 끝났으니, 선형 회귀 모델을 가져와 훈련하도록 합니다.
- 우선, scikit-learn 에서 LinearRegression 이란 이름의 선형 회귀 모델을 가져와 reg 라는 이름으로 정의하겠습니다.
- 훈련용 데이터를 이용하여 선형 회귀 모델을 훈련(fit)합니다.

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)

- 위를 통해 훈련이 완료되었습니다!
- 이제, 아래와 같이 훈련을 통해 계산된 계수(coefficient)들과 상수항(intercept)를 확인할 수 있습니다.

In [ ]:
print("the resulting coeffients of the linear regression (w1): \n", reg.coef_)
print("the resulting intercept of the linear regression (w0):", reg.intercept_)

### (4) Test and Evaluate the model
- 훈련된 모델을 이용하여 새로운 보스턴 주택 데이터가 주어졌을 때, 해당 데이터에 대한 주택 가격을 예측할 수 있습니다.
- 테스트 데이터에 대한 실제 y 값이 존재하므로, 실제값 y 와 predict 함수를 통해 예측된 값 y_hat 을 비교하여 모델을 평가할 수 있습니다.
- 이제, 훈련된 모델을 평가하기 위해 mean squared error ($=\sum_i (y_i-\hat{y}_i)^2/n$) 와 그것의 제곱근값, mean absolute error ($=\sum_i |y_i-\hat{y}_i|/n$), 그리고 $R^2$ 값을 구해봅시다.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_train_hat = reg.predict(X_train)
y_test_hat = reg.predict(X_test)

print('train RMSE : %.4f' % mean_squared_error(y_train, y_train_hat)**0.5)
print('train MAE : %.4f' % mean_absolute_error(y_train, y_train_hat))
print('train R-square : %.4f' % r2_score(y_train, y_train_hat))

print('test RMSE : %.4f' % mean_squared_error(y_test, y_test_hat)**0.5)
print('test MAE : %.4f' % mean_absolute_error(y_test, y_test_hat))
print('test R-square : %.4f' % r2_score(y_test, y_test_hat))

- MAE 가 3.1, RMSE 가 4.7 라는 것은, 평균적으로 약 3.1 정도의 오차를 가지고 예측을 수행하며, 예측 값과 실제 값 사이에는 약 4.7 정도의 평균적인 차이가 있다는 것을 의미합니다.
- $R^2$ 는 1에 가까울 수록 좋으며, 0.7 이상이면 예측이 어느 정도 된다고 해석합니다.
- 결과를 통해 훈련된 모델이 아주 정확하지는 않지만 적당히 예측력이 있다는 것을 알 수 있습니다.

## 2. Check the coefficients 
- 모델의 coef_ 와 intercept_ 값이 실제로 $(X^TX)^{-1}X^Ty$ 식에 의해 도출된 값과 일치하는 지 확인해보도록 하겠습니다.

In [ ]:
import numpy as np

In [ ]:
# intercept 를 포함한 coefficient matrix 를 구성한다.
X_new = np.ones((X_train.shape[0],X_train.shape[1]+1))
X_new[:,1:] = X_train
print(X_new.shape)

In [ ]:
inv = np.linalg.inv(np.dot(X_new.transpose(), X_new))
coeff = inv.dot(X_new.transpose()).dot(y_train)
print("coefficients (w1,w2,...) are : \n", coeff[1:])
print("intercept (w0) is : ", coeff[0])

- 위의 reg.coef_ 와 reg.intercept_ 에 의해 도출된 값과 동일하다는 것을 확인할 수 있습니다.